# TensorFlow Fundamentals

## Calculating Gradients

In a previous exercise, we practiced calculating partial derivatives on the following example:

$$ f(x,y) = \sqrt{x^2 + y^2}$$

$$\frac{\partial f}{\partial x} = \frac{x}{\sqrt{x^2 + y^2}}$$

$$\frac{\partial f}{\partial y} = \frac{y}{\sqrt{x^2 + y^2}}$$

### Question
Take a second to use a calculator or a sheet of paper to calcuate the following:

*   $\displaystyle f(3, 4) = ??$
    

*   $ \displaystyle \frac{\partial f(3, 4)}{\partial x} = ??$
    
   
*   $ \displaystyle \frac{\partial f(3, 4)}{\partial y} = ??$
   


### Answers:
* 
* 
* 

At its core, TensorFlow is a library for representing mathematical operations as graphical structures and automating the process of computing partial derivatives.  We can use it to write numpy-style mathematical operations:


In [ ]:
import tensorflow as tf
import numpy as np

def f(x, y):
    return tf.sqrt(x**2 + y**2)
    
x = tf.Variable(3, dtype=tf.float32)
y = tf.Variable(4, dtype=tf.float32)

print(f(x, y))

More interestingly, we can use a `GradientTape` to record methematical operations for automatic differentiation:

In [ ]:
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    tape.watch(y)
    fxy = f(x, y)
    
df_dx = tape.gradient(fxy, x)
df_dy = tape.gradient(fxy, y)

print("f(3, 4) = {:.5}".format(fxy))
print("df(3, 4)/dx = {:.5}".format(df_dx))
print("df(3, 4)/dy = {:.5}".format(df_dy))

Once we have the partial derivatives, we can minimize our function using gradient descent.  Use the cell below to find the x and y that minimize $ f(x,y) = \sqrt{x^2 + y^2}$.  You many need to experiment with the learning rate and the number of iterations.

In [ ]:
learning_rate = .01
iterations = 10

x = tf.Variable(3, dtype=tf.float32)
y = tf.Variable(4, dtype=tf.float32)

for iteration in range(iterations):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x)
        tape.watch(y)
        fxy = f(x, y)
    
    df_dx = tape.gradient(fxy, x)
    df_dy = tape.gradient(fxy, y)
    
    x.assign(x - learning_rate * df_dx)
    y.assign(y - learning_rate * df_dy)
    print('current "loss": {}'.format(fxy))

print("\nx: {}".format(x.numpy()))
print("y: {}".format(y.numpy()))
    

## DataSets

In machine learning it is often the case that training data is too large to fit in memory on a single machine.  We may also want to perform some pre-processing on the data as it is loaded.  The `tf.data.Dataset` class provides a standard interface of feeding data to a machine learning model.  `Dataset` objects act as Python generators. 

We can create a Dataset from a numpy array using the `from_tensor_slices` method:


In [ ]:

#Generate 6 random two-dimensional elements as column vectors:

features = np.round(np.random.random((6, 2, 1)), 2)
print("Numpy array of data:\n")
print(features)
  
# Build a dataset:

dataset = tf.data.Dataset.from_tensor_slices(features)

# iterate over the elements in the dataset:

print("\nIterate over the corresponding Dataset:\n")
for element in dataset:
    print(element)

## Batches

It is usually more efficent to process data in *batches* than individually. Here is an example of Tensorflow code that multiplies each element in our data set by an appropriately sized weight vector and sums the result:

In [ ]:
total = tf.Variable(np.zeros((1,1)))
weights = tf.Variable(np.random.random((2,1)))

for element in dataset:
    total = total + tf.matmul(tf.transpose(weights), element)
    print("Total so far: {}".format(total))

print("\nFinal Total: {}".format(total))

Instead of processing one data element per iteration, w can batch the dataset and process k-operations per iteration.  Many TensorFlow operators are "batch-aware" and will recognize that the first dimension corrsponds to the batch.  Let's look at a batched version of our dataset:

In [ ]:
dataset_batched = dataset.batch(2)
for batch in dataset_batched:
    print("Shape: {}\n".format(batch.shape))
    print("Elements:\n {}\n".format(batch))

In [ ]:
total = tf.Variable(np.zeros((1, 1)))

for batch in dataset_batched:
    batch_of_products = tf.matmul(tf.transpose(weights), batch)
    total = total + tf.reduce_sum(batch_of_products)
    print("Total so far: {}".format(total))

print("\nFinal Total: {}".format(total))